In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
from datetime import datetime
from time import sleep
from bs4 import BeautifulSoup
import re
import pandas as pd
import config

In [2]:
# disable notifications
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")

In [3]:
# open the facebook group (Student Housing in Waterloo)
# NOTE: replace path_to_chromedriver with your own path
driver = webdriver.Chrome(config.path_to_chromedriver, options=chrome_options) 
driver.get("https://www.facebook.com")

/var/folders/jf/mzp0fx9j59z164nkct92ngcc0000gn/T/ipykernel_4268/2693793491.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(config.path_to_chromedriver, options=chrome_options)


In [4]:
# Log in using my credentials
username = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

# my_username and my_password should be replaced with your own unique values
username.clear()
username.send_keys(config.my_username) 
password.clear()
password.send_keys(config.my_password)
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
sleep(8)

In [5]:
driver.get('https://www.facebook.com/groups/110354088989367')
sleep(8)

In [6]:
for j in range(0,10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(5)

In [7]:
soup = BeautifulSoup(driver.page_source, "html.parser")
all_posts = soup.find_all('div', attrs={'class':'du4w35lb k4urcfbm l9j0dhe7 sjgh65i0'})

In [8]:
len(all_posts) # 282 posts when scrolled 50 times - range(0, 50)

64

In [9]:
def get_bedrooms(my_str):
    my_str = my_str.lower()
    d = re.search('([^ \r\n]+) bedrooms?([\r\n]| |$)', my_str)
    if d:
        found = d.group(0)
        return found
    else:
        return 1

In [12]:
price_list = []
bedrooms_list = []

In [20]:
for post in all_posts: 
    price_text = post.find('span', attrs={'class': 'sqxagodl'}).find('div').getText()
    price = ''.join([n for n in price_text.split()[0] if n.isdigit()])
    if not price: 
        price = 0
#     print(price, type(price))
    
    message = post.find('div', attrs={'class':'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q'})
    if message: 
        message = str(message.getText())
    else:
        message = ''
        
    bedroom_text = get_bedrooms(message)
    bedroom = ''.join([n for n in str(bedroom_text).split()[0] if n.isdigit()])
    if not bedroom: 
        bedroom = 1
#     print('Number of bedrooms:', bedroom)
    
    price_list.append(int(price))
    bedrooms_list.append(int(bedroom))

In [21]:
price_list[:10], bedrooms_list[:10]

(['1200', '1', '750', '150', '750', '1', '', '800', '1', '1'],
 ['2', '2', '1', '1', 1, '2', '1', '1', '1', '1'])

In [22]:
len(price_list), len(bedrooms_list)

(198, 198)

In [28]:
data = {'price': price_list, 'bedroom': bedrooms_list}

In [29]:
df = pd.DataFrame(data)
df.to_excel('housing_list.xlsx', index=False)

In [ ]:
df = pd.DataFrame()

#### other things to consider: 
- number of bedrooms
- number of washrooms 
- number of roommates
- location (how far from uw?)
